In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
data='https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail/code'

In [4]:
od.download(data)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abhishek001chawla
Your Kaggle Key: ··········


100%|██████████| 503M/503M [00:05<00:00, 92.8MB/s]


In [5]:
import os

In [6]:
data_dir='./newspaper-text-summarization-cnn-dailymail'

In [7]:
os.listdir(data_dir)

['cnn_dailymail']

In [8]:
import pandas as pd
import numpy as np
df_train=pd.read_csv(r'/content/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv')

In [9]:
df_train.shape

(287113, 3)

In [10]:
df_train.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [11]:
df_validation=pd.read_csv(r'/content/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv')

In [12]:
df_validation.shape

(13368, 3)

In [13]:
df_test=pd.read_csv(r'/content/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv')

In [14]:
df_test.shape

(11490, 3)

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          287113 non-null  object
 1   article     287113 non-null  object
 2   highlights  287113 non-null  object
dtypes: object(3)
memory usage: 6.6+ MB


In [16]:
df_train.columns

Index(['id', 'article', 'highlights'], dtype='object')

In [17]:
df_train.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [18]:
df_train.duplicated().any()

False

In [19]:
training=df_train['article'][:500].replace("'","")
testing=df_train['highlights'][:500].replace("'","")

In [20]:
import tensorflow as tf
tokenizer=tf.keras.preprocessing.text.Tokenizer()

In [21]:
def token_news(sent):
  tokenizer.fit_on_texts([sent])
  seq=tokenizer.texts_to_sequences([sent])
  seq_len=50
  pad_seq=tf.keras.preprocessing.sequence.pad_sequences(seq,maxlen=seq_len)
  return pad_seq

In [22]:
import pandas as pd
import numpy as np

In [23]:
training=training.apply(token_news,tokenizer)

In [24]:
training

0      [[7, 1, 5, 34, 86, 87, 1, 88, 89, 90, 91, 92, ...
1      [[5, 277, 4, 278, 3, 92, 93, 279, 3, 92, 93, 5...
2      [[101, 34, 38, 2, 20, 238, 551, 10, 1, 68, 28,...
3      [[5, 75, 144, 790, 15, 791, 792, 793, 148, 794...
4      [[1049, 122, 1, 1050, 1, 88, 340, 367, 108, 10...
                             ...                        
495    [[4104, 35, 14, 5369, 25856, 663, 462, 5, 1, 3...
496    [[1321, 731, 6, 1, 148, 5, 10658, 31, 25916, 1...
497    [[5040, 47, 1, 9212, 62, 1857, 33, 856, 3337, ...
498    [[1418, 106, 1, 195, 76, 5, 24, 14762, 177, 1,...
499    [[3140, 39, 7647, 939, 5464, 1067, 2, 9328, 58...
Name: article, Length: 500, dtype: object

In [25]:
testing=testing.apply(token_news,tokenizer)

In [26]:
testing

0      [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1      [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2      [[262, 777, 6, 8577, 4890, 5, 7145, 1771, 247,...
3      [[14533, 26044, 12001, 117, 909, 436, 22, 1016...
4      [[55, 1928, 62, 844, 5, 1, 315, 1017, 6, 1248,...
                             ...                        
495    [[2477, 16, 1796, 65, 2, 782, 532, 92, 9, 37, ...
496    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65...
497    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58, 11298, ...
498    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
499    [[2, 383, 4, 3, 219, 74, 6, 469, 1324, 2131, 7...
Name: highlights, Length: 500, dtype: object

In [27]:
from tensorflow.keras.models import Model

In [28]:
vocab_size=10000
embedding_dim=200
seq_len=50
input=tf.keras.Input(shape=seq_len)
ehs=[]
e_layer=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim)(input)
e_lstm=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1600,return_sequences=True))(e_layer)
encoded=tf.keras.layers.Concatenate()([e_lstm[:,-1,:],e_lstm[:,0,:]])
model=tf.keras.Model(inputs=input,outputs=encoded)

In [29]:
def encoder(a,ta,model):
  o=tf.keras.optimizers.Adam(learning_rate=0.01)
  for i in range(len(a)):
    with tf.GradientTape() as tape:
      result=model(a[i],training=True)
      test_result=model(ta[i],training=True)
      loss=tf.keras.losses.mean_squared_error(result,test_result)

      ehs.append(result)
      g=tape.gradient(loss,model.trainable_variables)
      o.apply_gradients(zip(g,model.trainable_variables))
      return ehs

In [47]:
def Decoder(eh,target_tokens,model):
  loss=0
  seq_len=50
  hidden_size=128
  emb_dim=128
  vocab_size=10000
  c_vector=tf.zeros((128,50))
  d_state=tf.zeros((128,50))
  learning_rate=0.01
  o=tf.keras.optimizers.Adam(learning_rate=0.01)
  v = tf.random.normal(shape=(hidden_size,50))
  Wh = tf.random.normal(shape=(hidden_size, 50))
  Ws = tf.random.normal(shape=(hidden_size, 50))
  battn = tf.random.normal(shape=(hidden_size,50))
  weights= tf.random.normal(shape=(100, 1))
  bias=tf.random.uniform([])

  for t in range(len(target_tokens)):
    input=tf.keras.Input(shape=(seq_len))
    e_layer=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=emb_dim)(input)
    model=tf.keras.Model(inputs=input,outputs=e_layer)
    o_e=model.predict(target_tokens[t])
    attention_score=[]
    for i in range(len(ehs)):
      ehs[i]=tf.reshape(ehs[i],(128,50))
      eti=v*tf.nn.tanh(Wh*ehs[i]+Ws*d_state+battn)
      attention_score.append(eti)

    ad=tf.nn.softmax(attention_score)
    for i in range(len(ehs)):
      c_vector+=ad[i]*ehs[i]

    o_e=tf.reshape(o_e,((128,50,1)))
    o_e=tf.squeeze(o_e,axis=-1)
    c_i=tf.expand_dims(tf.concat([c_vector,o_e],axis=-1),axis=0)
    oe1=tf.matmul(c_i,weights)+bias
    vocab_dist=tf.nn.softmax(oe1,axis=1)
    hpi=np.argmax(vocab_dist)
    twp=vocab_dist[0,hpi,0]
    timeless_loss=-tf.math.log(twp)
    d_state=(d_state+c_vector)/2
    d_state=tf.reshape(d_state,(128,50))

    loss+=timeless_loss
    m_loss=loss/(t+1)

    v+=learning_rate*m_loss*tf.nn.tanh(Wh*ehs[i]+Ws*d_state+battn)
    Ws+=learning_rate*m_loss*d_state
    Wh=Wh+learning_rate*m_loss*ehs[i]
    battn+=learning_rate*m_loss
    bias+=learning_rate*m_loss
    weights+=learning_rate*m_loss*weights
    d_state=(d_state+c_vector)/2
    d_state = tf.reshape(d_state, (128, 50))


  return [v,Ws,Wh,battn,weights,bias]

In [31]:
eh=encoder(training,testing,model)

In [32]:
is1=tf.keras.Input(shape=(128,100))
l2=tf.keras.layers.Dense(1)(is1)
model=tf.keras.Model(inputs=is1,outputs=l2)

In [33]:
testing

0      [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1      [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2      [[262, 777, 6, 8577, 4890, 5, 7145, 1771, 247,...
3      [[14533, 26044, 12001, 117, 909, 436, 22, 1016...
4      [[55, 1928, 62, 844, 5, 1, 315, 1017, 6, 1248,...
                             ...                        
495    [[2477, 16, 1796, 65, 2, 782, 532, 92, 9, 37, ...
496    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65...
497    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 58, 11298, ...
498    [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
499    [[2, 383, 4, 3, 219, 74, 6, 469, 1324, 2131, 7...
Name: highlights, Length: 500, dtype: object

In [34]:
eh

[<tf.Tensor: shape=(1, 6400), dtype=float32, numpy=
 array([[-0.00229631, -0.00151995,  0.00104182, ..., -0.0030432 ,
          0.0023452 ,  0.00141003]], dtype=float32)>]

In [35]:
model

In [36]:
type(testing)

pandas.core.series.Series

In [37]:
type(eh)

list

In [ ]:
v, Ws, Wh, battn, weights, bias = decoder(eh,testing,model)

In [49]:
def tests(para,v,ws,wh,battn,weights,model,bias,tokenizer,l=50):
  tokenizer.fit_on_texts([para])
  seq=tokenizer.texts_to_sequences([para])
  seq_len=50
  pad_seq=tf.keras.preprocessing.sequence.pad_sequences(seq,maxlen=seq_len)
  t_para=token_news(para)
  ehs=[model.predict(t_para)]
  summary=[]
  hs=128
  ed=50
  c_vector=tf.zeros((128,50))
  d_state=tf.zeros((128,50))
  ct=np.zeros((128))
  for j in range(l):
    i=tf.keras.Input(shape=(seq_len))
    el=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=ed)(i)
    model=tf.keras.Model(inputs=i,outputs=el)
    oe=model.predict(ct,verbose=0)

    attention_score=[]
    for i in range(len(ehs)):
      ehs[i]=tf.reshape(ehs[i],(128,50))
      eti=v*tf.nn.tanh(wh*ehs[i]+ws*d_state+battn)
      attention_score.append(eti)

    ad=tf.nn.softmax(attention_score)
    for i in range(len(ehs)):
      c_vector+=ad[i]*ehs[i]

    oe=tf.reshape(oe,(128,50,1))
    oe=tf.squeeze(oe,axis=-1)
    ci=tf.expand_dims(tf.concat([c_vector,oe]),axis=0)
    oe1=tf.matmul(ci,weights)+bias
    vd=tf.nn.softmax(oe1,axis=1)
    hpi=np.argmax(vd)
    ct[j]=hpi
    d_state=(d_state+c_vector)/2
    d_state=tf.reshape(d_state,(128,50))
    summary.append(ct)

  ttw={tokenizer.word_index[i]:i for i in tokenizer.word_index.keys()}
  ttw.update({0.0:'',127.0:''})
  return (' '.join([ttw[i] for i in ct]))



In [50]:
df_test

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [51]:
test=df_test['article'][1:3].replace("'","")

In [ ]:
for i in test:
  print(i+'\n')
  print(tests(i,v,Ws,Wh,battn,weights,model,bias,tokenizer,l=50))